<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/deteccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os

if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

Cargar las librerias que vamos a utilizar

In [10]:
import sys
import subprocess

def instalar_si_falta(paquete, nombre_import=None):
    if nombre_import is None:
        nombre_import = paquete
    try:
        __import__(nombre_import)
    except ImportError:
        print(f"Instalando {paquete}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", paquete])

# Lista de paquetes a instalar/importar
paquetes = [
    ("cv2", "opencv-python"),
    ("numpy", "numpy"),
    ("PIL", "Pillow"),
    ("sklearn", "scikit-learn"),
    ("scipy", "scipy")
]

for nombre_import, paquete in paquetes:
    instalar_si_falta(paquete, nombre_import)

# Ahora puedes importar normalmente
import cv2
import math
import numpy as np
import os
import pickle
import random
import sys
from sklearn.manifold import Isomap
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
import scipy

# Detectamos caras en imagenes
## La carpeta de origen se llama Detectar
## y las guardamos en Detectadas

In [16]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Definicion de ruta de conexion - Origen - Destino imagenes

img_size=(120,120)

ruta_entrada = '/content/drive/MyDrive/fotos'
ruta_salida = f'/content/drive/MyDrive/Detectadas'


# prompt: verificar si la carpeta en "ruta_salida" existe. Si no existe, crearla

if not os.path.exists(ruta_salida):
  os.makedirs(ruta_salida)

def procesar_archivos_en_carpetas(ruta_principal,ruta_final, imgsize):
  """Recorre las carpetas dentro de la ruta principal y procesa los archivos.

  Args:
    ruta_principal: La ruta de la carpeta principal.
  """



  x_inicial = 0
  y_inicial = 0
  alto = imgsize[1]
  ancho = imgsize[0]
  img_size = (ancho ,alto)



  print(f"Ruta principal: {ruta_principal}")
  for carpeta_actual, _, archivos in os.walk(ruta_principal):
    print(f"Carpeta actual: {carpeta_actual}")
    for archivo in archivos:
      print(f"Archivo: {archivo}")
      ruta_completa = os.path.join(carpeta_actual, archivo)
      try:

            img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)

            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

            if img is None:
                print(f"⚠️ No se pudo leer: {ruta_completa}")
                continue

            # Detectar rostros
            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=10, minSize=(110, 110))

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]  # Recortar rostro
                face_resized = cv2.resize(face, img_size)  # Redimensionar

                face_cutted = face_resized[y_inicial:y_inicial+alto, x_inicial:x_inicial+ancho]

                # Crear carpeta de salida manteniendo la estructura original
                relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                output_folder = os.path.join(ruta_salida, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                # Guardar rostro detectado
                output_path = os.path.join(output_folder, f"{os.path.splitext(archivo)[0]}_face{i}.jpg")
                cv2.imwrite(output_path, face_cutted)
                print(f"✅ Guardado: {output_path}")


      except Exception as e:
          print(f"❌ Error procesando {ruta_completa}: {e}")


      except Exception as e:
        print(f"Error al procesar el archivo {ruta_completa}: {e}")

In [17]:
# Deteccion de caras (escala de grises, recortes y escalado)
print("Detectando Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección inicial terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")

Detectando Caras ...
Ruta principal: /content/drive/MyDrive/fotos
Carpeta actual: /content/drive/MyDrive/fotos
Carpeta actual: /content/drive/MyDrive/fotos/Federico
Archivo: IMG_1564.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1564_face0.jpg
Archivo: 1742907850028.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/1742907850028_face0.jpg
Archivo: IMG_1562.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1562_face0.jpg
Archivo: IMG_1182.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1182_face0.jpg
Archivo: IMG_1568.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1568_face0.jpg
Archivo: IMG_1189.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1189_face0.jpg
Archivo: IMG_1192.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1192_face0.jpg
Archivo: 20250321_140311.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/20250321_140311_face0.jpg
Archivo: IMG_1078.JPG
✅ Guardado: /content/

In [5]:
# Version nueva optimizada que reemplazaria a lo anterior

La celda de código que sigue a esta celda markdown implementa un pipeline de detección y recorte de rostros en imágenes almacenadas en Google Drive, utilizando OpenCV en Python. El proceso comienza asegurando que Google Drive esté montado y que existan las carpetas de entrada y salida necesarias. Luego, se inicializa el clasificador de rostros de OpenCV con un modelo Haar Cascade preentrenado.

La función principal, `procesar_archivos_en_carpetas`, recorre recursivamente todas las carpetas y archivos dentro de la carpeta de entrada especificada. Para cada imagen encontrada, la lee en escala de grises y normaliza sus valores de intensidad. Si la imagen se puede leer correctamente, utiliza el clasificador para detectar rostros. Por cada rostro detectado, recorta la región correspondiente, la redimensiona a un tamaño estándar y realiza un recorte adicional para eliminar bordes no deseados. Finalmente, guarda cada rostro procesado en la carpeta de salida, manteniendo la estructura de subcarpetas original y agregando un sufijo al nombre del archivo para identificar los distintos rostros detectados en una misma imagen.

El código imprime mensajes informativos durante el procesamiento, indicando el progreso y cualquier error que ocurra al leer o procesar archivos. Al finalizar, recuerda al usuario que debe revisar manualmente la carpeta de salida para eliminar recortes que no correspondan a rostros reales, ya que el detector puede generar falsos positivos. Este pipeline es útil para preparar conjuntos de datos de rostros para tareas de entrenamiento en reconocimiento facial o aprendizaje automático.

In [6]:
import os
import math
import cv2

# Montar Google Drive si es necesario
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

# Verificar si la carpeta de salida existe, si no, crearla
img_size = (80, 80)
ruta_entrada = '/content/drive/MyDrive/Detectadas'
ruta_salida = '/content/drive/MyDrive/Detectadas2'

if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Inicializar el clasificador de rostros de OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def procesar_archivos_en_carpetas(ruta_principal, ruta_final, imgsize):
    """
    Recorre las carpetas dentro de la ruta principal y procesa los archivos.
    Args:
        ruta_principal: La ruta de la carpeta principal.
        ruta_final: La ruta de la carpeta de salida.
        imgsize: Tupla con el tamaño de la imagen de salida.
    """
    fraccion = 8
    x_inicial = imgsize[0] // (fraccion - 2)
    y_inicial = imgsize[1] // (fraccion - 2)
    alto = imgsize[1]
    ancho = imgsize[0]
    multip = fraccion / (fraccion - 2)
    img_size = (math.floor(ancho * multip), math.floor(alto * multip))

    print(f"Ruta principal: {ruta_principal}")
    for carpeta_actual, _, archivos in os.walk(ruta_principal):
        print(f"Carpeta actual: {carpeta_actual}")
        for archivo in archivos:
            print(f"Archivo: {archivo}")
            ruta_completa = os.path.join(carpeta_actual, archivo)
            try:
                img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"⚠️ No se pudo leer: {ruta_completa}")
                    continue

                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Detectar rostros
                faces = face_cascade.detectMultiScale(
                    img,
                    scaleFactor=1.1,
                    minNeighbors=10,
                    minSize=(110, 110)
                )

                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y + h, x:x + w]  # Recortar rostro
                    face_resized = cv2.resize(face, img_size)  # Redimensionar
                    face_cutted = face_resized[
                        y_inicial:y_inicial + alto,
                        x_inicial:x_inicial + ancho
                    ]

                    # Crear carpeta de salida manteniendo la estructura original
                    relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                    output_folder = os.path.join(ruta_salida, relative_path)
                    os.makedirs(output_folder, exist_ok=True)

                    # Guardar rostro detectado
                    output_path = os.path.join(
                        output_folder,
                        f"{os.path.splitext(archivo)[0]}_face{i}.jpg"
                    )
                    cv2.imwrite(output_path, face_cutted)
                    print(f"✅ Guardado: {output_path}")

            except Exception as e:
                print(f"❌ Error procesando {ruta_completa}: {e}")

print("Detectar Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")

Detectar Caras ...
Ruta principal: /content/drive/MyDrive/fotos
Carpeta actual: /content/drive/MyDrive/fotos
Carpeta actual: /content/drive/MyDrive/fotos/Federico
Archivo: IMG_1564.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1564_face0.jpg
Archivo: 1742907850028.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/1742907850028_face0.jpg
Archivo: IMG_1562.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1562_face0.jpg
Archivo: IMG_1182.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1182_face0.jpg
Archivo: IMG_1568.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1568_face0.jpg
Archivo: IMG_1189.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1189_face0.jpg
Archivo: IMG_1192.JPG
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/IMG_1192_face0.jpg
Archivo: 20250321_140311.jpg
✅ Guardado: /content/drive/MyDrive/Detectadas/Federico/20250321_140311_face0.jpg
Archivo: IMG_1078.JPG
✅ Guardado: /content/dr

In [6]:
import os
import math
import cv2

# Montar Google Drive si es necesario
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')

# Verificar si la carpeta de salida existe, si no, crearla
img_size = (80, 80)
ruta_entrada = '/content/drive/MyDrive/Aumentadas'
ruta_salida = '/content/drive/MyDrive/Aumentadas'

if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Inicializar el clasificador de rostros de OpenCV
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def procesar_archivos_en_carpetas(ruta_principal, ruta_final, imgsize):
    """
    Recorre las carpetas dentro de la ruta principal y procesa los archivos.
    Args:
        ruta_principal: La ruta de la carpeta principal.
        ruta_final: La ruta de la carpeta de salida.
        imgsize: Tupla con el tamaño de la imagen de salida.
    """
    fraccion = 8
    x_inicial = imgsize[0] // (fraccion - 2)
    y_inicial = imgsize[1] // (fraccion - 2)
    alto = imgsize[1]
    ancho = imgsize[0]
    multip = fraccion / (fraccion - 2)
    img_size = (math.floor(ancho * multip), math.floor(alto * multip))

    print(f"Ruta principal: {ruta_principal}")
    for carpeta_actual, _, archivos in os.walk(ruta_principal):
        print(f"Carpeta actual: {carpeta_actual}")
        for archivo in archivos:
            print(f"Archivo: {archivo}")
            ruta_completa = os.path.join(carpeta_actual, archivo)
            try:
                img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"⚠️ No se pudo leer: {ruta_completa}")
                    continue

                img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

                # Detectar rostros
                faces = face_cascade.detectMultiScale(
                    img,
                    scaleFactor=1.1,
                    minNeighbors=10,
                    minSize=(110, 110)
                )

                for i, (x, y, w, h) in enumerate(faces):
                    face = img[y:y + h, x:x + w]  # Recortar rostro
                    face_resized = cv2.resize(face, img_size)  # Redimensionar
                    face_cutted = face_resized[
                        y_inicial:y_inicial + alto,
                        x_inicial:x_inicial + ancho
                    ]

                    # Crear carpeta de salida manteniendo la estructura original
                    relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                    output_folder = os.path.join(ruta_salida, relative_path)
                    os.makedirs(output_folder, exist_ok=True)

                    # Guardar rostro detectado
                    output_path = os.path.join(
                        output_folder,
                        f"{os.path.splitext(archivo)[0]}_face{i}.jpg"
                    )
                    cv2.imwrite(output_path, face_cutted)
                    print(f"✅ Guardado: {output_path}")

            except Exception as e:
                print(f"❌ Error procesando {ruta_completa}: {e}")

print("Detectar Caras ...")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Detección terminada ...")
print("Para entrenamiento, hay que limpiar manualmente la carpeta de salida, para eliminar los recortes que no son rostros.")